In [24]:
 
from transformers import pipeline
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig

label2id = {"Left": 0, "Center": 1, "Right": 2}
id2label = {0: "Left", 1: "Center", 2: "Right"}

config = AutoConfig.from_pretrained("premsa/political-bias-prediction-allsides-DeBERTa", label2id=label2id, id2label=id2label)
political_model = AutoModelForSequenceClassification.from_pretrained("premsa/political-bias-prediction-allsides-DeBERTa", config=config)
# political_model = AutoModelForSequenceClassification.from_pretrained("premsa/political-bias-prediction-allsides-DeBERTa")#, config=config)
political_tokenizer = AutoTokenizer.from_pretrained("premsa/political-bias-prediction-allsides-DeBERTa")




/Users/jperez/miniconda3/envs/cellm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [42]:
texts = [
    "Trans rights are the most important issue today!",
    "I am a left winger!",
    "I am a communist!",
    "I love Biden!",
    "I love the democrats!",
    "I love the republicans!",
    "I love the bible!",
    "I love Trump!",
    "Enough with the immigrants!",
    "If you're gay just stay away from me!",
    "I don't care if you're gay but stay away from me!",
    "I like apples.",
    "Happy New Year!",
    "I am a right winger!",
"The LGBTQ community around the world will once again know that America stands with them and will protect their rights.",
"The LGBTQ community around the world will once again know that America does not stand with them and will never protect their rights."
]

political_classifier = pipeline("text-classification", model=political_model, tokenizer=political_tokenizer, top_k=None)
results = political_classifier(texts)

zero_shot = pipeline(model="facebook/bart-large-mnli", task="zero-shot-classification")
results_zero = zero_shot(texts, candidate_labels=["Left", "Center", "Right"])

for text, result, result_zero in zip(texts, results, results_zero):
    print("debert:",result)
    print("zeroshot:",result_zero)
    print(f"Text: {text}")
    print(f"Label: {result['label']}")
    print(f"Score: {result['score']}")
    print()

debert: [{'label': 'Left', 'score': 0.5068954229354858}, {'label': 'Center', 'score': 0.26981568336486816}, {'label': 'Right', 'score': 0.223288893699646}]
zeroshot: {'sequence': 'Trans rights are the most important issue today!', 'labels': ['Right', 'Center', 'Left'], 'scores': [0.5743761658668518, 0.2331705242395401, 0.19245336949825287]}
Text: Trans rights are the most important issue today!


TypeError: list indices must be integers or slices, not str

In [37]:
from dataset_utils import load_human_dataset

dataset_name = "twitter"
data = load_human_dataset(dataset_name)


Loading dataset m-newhauser/senator-tweets
Loading dataset from cache in: data/dataset_cache/twitter_lean_None.save


In [41]:
for i in range(20):
    text = data[i]['text']
    result = political_classifier(text)
    print(f"Text: {text}")
    print(result)
    # if result['label'] != 'Right' and result['label'] != 'Left':
    #     print(result)

       
 

Text: Happy th birthday to the @USNavy! The strength, dedication, and skill of our Sailors including those at Portsmouth Naval Shipyard help keep this country safe, secure, and free. Today we recognize and celebrate their incredible service. #246NavyBirthday
[[{'label': 'Left', 'score': 0.40177229046821594}, {'label': 'Center', 'score': 0.15773463249206543}, {'label': 'Right', 'score': 0.44049301743507385}]]
Text: The greatest generation's investment in infrastructure made us the envy of the world. But now we've gone almost an entire lifetime without making any significant investments in the NEXT generation of American infrastructure. It's time for that to change.
[[{'label': 'Left', 'score': 0.6259058713912964}, {'label': 'Center', 'score': 0.1849103420972824}, {'label': 'Right', 'score': 0.1891838014125824}]]
Text: Thanks to @SenTedCruz and  @SenatorWarnock, the Infrastructure Investment and Jobs Act authorizes the Interstate East-West corridor across TX, LA, MS, AL, and GA. Working 

In [28]:
from transformers import AutoModel, AutoTokenizer, pipeline
import torch

# choose GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# select mode path here
pretrained_LM_path = "kornosk/polibertweet-mlm"

# load model
tokenizer = AutoTokenizer.from_pretrained(pretrained_LM_path)
model = AutoModel.from_pretrained(pretrained_LM_path)

# fill mask
example = "Trump is the <mask> of USA"
fill_mask = pipeline('fill-mask', model=pretrained_LM_path, tokenizer=tokenizer)

outputs = fill_mask(example)
print(outputs)

# see embeddings
inputs = tokenizer(example, return_tensors="pt")
outputs = model(**inputs)
print(outputs)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0
Some weights of RobertaModel were not initialized from the model checkpoint at kornosk/polibertweet-mlm and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'score': 0.2734937071800232, 'token': 1652, 'token_str': 'president', 'sequence': 'Trump is the president of USA'}, {'score': 0.17018833756446838, 'token': 1073, 'token_str': 'President', 'sequence': 'Trump is the President of USA'}, {'score': 0.06884991377592087, 'token': 4851, 'token_str': 'enemy', 'sequence': 'Trump is the enemy of USA'}, {'score': 0.03327561169862747, 'token': 28023, 'token_str': 'savior', 'sequence': 'Trump is the savior of USA'}, {'score': 0.023005496710538864, 'token': 2222, 'token_str': 'leader', 'sequence': 'Trump is the leader of USA'}]
BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-4.1815e-03, -1.1005e-01,  4.1521e-01,  ..., -2.9144e-01,
           7.2262e-03, -3.2134e-01],
         [-1.3225e-01, -2.5746e-01,  2.0411e-01,  ..., -1.0303e-01,
           4.5471e-01, -7.4095e-01],
         [-7.4149e-02, -2.1878e-01,  3.2496e-01,  ...,  4.4154e-02,
           1.9670e-01, -6.2388e-02],
         ...,
         [-3.0614e-01, -4.0273e-02, 

In [33]:
for text in texts:
    example = "A polticial from the <mask> party said: " + text

    fill_mask = pipeline('fill-mask', model=pretrained_LM_path, tokenizer=tokenizer)
    

    outputs = fill_mask(example)

    #find most probable word between democratic and Republican
    for output in outputs:
        #remove capitalization
        output['token_str'] = output['token_str'].lower()
        if output['token_str'] == 'democratic':
            print("democratic")
            print(output)
            break
        elif output['token_str'] == 'republican':
            print("republican")
            print(output)
            break

    # print(outputs)

    # # see embeddings
    # inputs = tokenizer(example, return_tensors="pt")
    # outputs = model(**inputs)
    # print(outputs)

democratic
{'score': 0.0841638371348381, 'token': 6085, 'token_str': 'democratic', 'sequence': 'A polticial from the Democratic party said : Trans rights are the most important issue today!'}
democratic
{'score': 0.07604547590017319, 'token': 11437, 'token_str': 'democratic', 'sequence': 'A polticial from the democratic party said : I am a left winger!'}
democratic
{'score': 0.06510443240404129, 'token': 11437, 'token_str': 'democratic', 'sequence': 'A polticial from the democratic party said : I am a communist!'}
democratic
{'score': 0.17135824263095856, 'token': 6085, 'token_str': 'democratic', 'sequence': 'A polticial from the Democratic party said : I love Biden!'}
democratic
{'score': 0.06296046078205109, 'token': 11437, 'token_str': 'democratic', 'sequence': 'A polticial from the democratic party said : I love the democrats!'}
republican
{'score': 0.034486982971429825, 'token': 4265, 'token_str': 'republican', 'sequence': 'A polticial from the Republican party said : I love the r

In [39]:
for d in data:
    example = "A poltician from the <mask> party said: " + d['text']
    label = d['Political Lean']

    fill_mask = pipeline('fill-mask', model=pretrained_LM_path, tokenizer=tokenizer)
    

    outputs = fill_mask(example)

    #find most probable word between democratic and Republican
    for output in outputs:
        #remove capitalization
        output['token_str'] = output['token_str'].lower()
        if output['token_str'] == 'democratic':
            print("democratic: ", label)
            print(output)
            break
        elif output['token_str'] == 'republican':
            print("republican: ", label)
            print(output)
            break

    # print(outputs)

    # # see embeddings
    # inputs = tokenizer(example, return_tensors="pt")
    # outputs = model(**inputs)
    # print(outputs)

/Users/jperez/miniconda3/envs/cellm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


democratic:  Conservative
{'score': 0.07064133882522583, 'token': 6085, 'token_str': 'democratic', 'sequence': 'A polticial from the Democratic party said : Thanks to @USER and @USER, the Infrastructure Investment and Jobs Act authorizes the Interstate East-West corridor across TX, LA, MS, AL, and GA. Working together to deliver real infrastructure!'}
republican:  Conservative
{'score': 0.04800720512866974, 'token': 4265, 'token_str': 'republican', 'sequence': 'A polticial from the Republican party said : Today were celebrating years of the Hoosier state. Happy birthday Indiana!'}
democratic:  Liberal
{'score': 0.0957123190164566, 'token': 6085, 'token_str': 'democratic', 'sequence': 'A polticial from the Democratic party said : The #ForthePeopleAct includes reforms that are popular with Americans no matter what party theyre from : Expanded early voting Automatic voter registration Public disclosure of campaign contributions So, lets get it done.'}
republican:  Liberal
{'score': 0.1272

KeyboardInterrupt: 

In [53]:
#check data
import pickle


with open('data_twitter_DeBertA_1000samples_seed42.pkl', 'rb') as f:
    data = pickle.load(f)

print(data.keys())

for text, predcited_label, actual_label in zip(data['texts'], data['predcited_labels'], data['actual_labels']):
    if predcited_label != actual_label: 
        print("Text:",text)
        print("predcited_label:",predcited_label)
        print("actual_label: ",actual_label)
        print()


dict_keys(['texts', 'predcited_labels', 'actual_labels'])
Text: Idahoans deserve serious, bipartisan solutions focusing on their most urgent needs: getting or keeping their kids in school getting workers safely back on the job keeping small businesses doors open
predcited_label: Liberal
actual_label:  Conservative

Text: Men and women in law enforcement make tremendous sacrifices every day to protect and serve the public, some paying the ultimate sacrifice. Our fallen heroes will never be forgotten. #PeaceOfficersMemorialDay #PoliceWeek
predcited_label: Liberal
actual_label:  Conservative

Text: As Utah wildfires become more destructive, we must not adhere to fire policies of the past. Our bipartisan infrastructure bill establishes a commission to study and recommend fire prevention, mitigation, management, and rehabilitation policies for our forests and grasslands.
predcited_label: Liberal
actual_label:  Conservative

Text: The press has reported extensively on the so-called voter sup

In [ ]:
## plot accuracy as a funcion of nsamples, for each model
import matplotlib.pyplot as plt
import pickle

models = ['zero_shot', 'DeBertA', 'zero_shot_multi_label', 'gpt-4o', 'gpt-4o-mini', 'Llama-3.1-8B-Instruct-Turbo', 'Llama-3.1-70B-Instruct-Turbo', 'Qwen2.5-72B-Instruct-Turbo']


In [55]:
## get logits for L and R

import transformers


model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto"
)

print(pipeline.tokenizer.encode("democratic"))

/Users/jperez/miniconda3/envs/cellm/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/jperez/miniconda3/envs/cellm/lib/python3.10/site-packages/accelerate/utils/modeling.py:1341: UserWarning: Current model requires 121638528 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:06<00:00,  1.68s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


[128000, 28531, 38341]


In [56]:
print(pipeline.tokenizer.encode("L"))

[128000, 43]


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"



model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
inputs = tokenizer(["The full name of Donald is Donald"], return_tensors="pt")

summary_ids = model.generate(inputs["input_ids"], max_new_tokens=4)
print(tokenizer.batch_decode(summary_ids, skip_special_tokens=True)[0])
# The full name of Donald is Donald J. Trump Jr

# Now let's control generation through a bias. Please note that the tokenizer is initialized differently!
tokenizer_with_prefix_space = AutoTokenizer.from_pretrained("openai-community/gpt2", add_prefix_space=True)


def get_tokens(word):
    return tokenizer_with_prefix_space([word], add_special_tokens=False).input_ids[0]


# If we add a negative bias without beam search, it may become "stuck" in a prefix without good continuations
sequence_bias = [get_tokens("Trump"), -10.0]
biased_ids = model.generate(inputs["input_ids"], max_new_tokens=4, sequence_bias=sequence_bias)
print(tokenizer.batch_decode(biased_ids, skip_special_tokens=True)[0])
# The full name of Donald is Donald J. Donald,

biased_ids = model.generate(inputs["input_ids"], max_new_tokens=4, num_beams=4, sequence_bias=sequence_bias)
print(tokenizer.batch_decode(biased_ids, skip_special_tokens=True)[0])
# The full name of Donald is Donald Rumsfeld,

# We can also add a positive bias to nudge the model towards specific tokens or continuations
sequence_bias = [get_tokens("Donald Duck"), 10.0]
biased_ids = model.generate(inputs["input_ids"], max_new_tokens=4, num_beams=4, sequence_bias=sequence_bias)
print(tokenizer.batch_decode(biased_ids, skip_special_tokens=True)[0])
# The full name of Donald is Donald Duck.

/Users/jperez/miniconda3/envs/cellm/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/jperez/miniconda3/envs/cellm/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 4/4 [01:00<00:00, 15.03s/it]
/Users/jperez/miniconda3/envs/cellm/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word

KeyboardInterrupt: 